# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,12.79,63,100,1.79,NZ,1731958536
1,1,college,64.8569,-147.8028,-21.03,77,75,0.00,US,1731958538
2,2,talnakh,69.4865,88.3972,-8.56,96,100,0.94,RU,1731958539
3,3,thompson,55.7435,-97.8558,0.09,93,100,1.03,CA,1731958540
4,4,edinburgh of the seven seas,-37.0676,-12.3116,14.14,74,12,8.43,SH,1731958541


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:


# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng',         # Longitude as x-axis
    'Lat',         # Latitude as y-axis
    size='Humidity',  # Point size determined by Humidity
    geo=True,         # Enables a geographic map
    tiles='OSM',      # Use OpenStreetMap tiles as background
    color='blue',     # Point color
    alpha=0.6,        # Transparency for the points
    title="City Locations with Humidity Represented by Point Size",
    frame_width=800,
    frame_height=500
)
# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &  # Max Temp between 21 and 27
    (city_data_df['Wind Speed'] < 4.5) &                                      # Wind speed less than 4.5 m/s
    (city_data_df['Cloudiness'] == 0)                                          # Zero cloudiness
]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,ar ruways,24.1103,52.7306,23.85,60,0,3.01,AE,1731958601
82,82,mirpur sakro,24.5474,67.6277,25.38,35,0,2.16,PK,1731958630
263,263,cabo san lucas,22.8909,-109.9124,22.64,42,0,4.12,MX,1731958853
272,272,agat village,13.3886,144.6585,25.98,89,0,3.60,GU,1731958863
282,282,bandar abbas,27.1865,56.2808,22.99,78,0,3.09,IR,1731958874


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
57,ar ruways,AE,24.1103,52.7306,60,None
82,mirpur sakro,PK,24.5474,67.6277,35,None
263,cabo san lucas,MX,22.8909,-109.9124,42,None
272,agat village,GU,13.3886,144.6585,89,None
282,bandar abbas,IR,27.1865,56.2808,78,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000  # Radius in meters (10,000 meters)
params = {
    "categories": "accommodation.hotel",  # Searching for hotels
    "limit": 1,                          # Limit to one hotel per city
    "apiKey": geoapify_key                # API key for Geoapify
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"  # Filter for a 10,000 meter radius around the city
    params["bias"] = f"point:{lon},{lat}"  # Bias the search to the city's coordinates

    # Set base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
ar ruways - nearest hotel: No hotel found
mirpur sakro - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
agat village - nearest hotel: No hotel found
bandar abbas - nearest hotel: No hotel found
khairpur - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
at taraf - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
xai-xai - nearest hotel: No hotel found
ilha de mocambique - nearest hotel: No hotel found
palatka - nearest hotel: No hotel found
south hedland - nearest hotel: No hotel found
i-n-salah - nearest hotel: No hotel found
al muwayh - nearest hotel: No hotel found
piti village - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
57,ar ruways,AE,24.1103,52.7306,60,No hotel found
82,mirpur sakro,PK,24.5474,67.6277,35,No hotel found
263,cabo san lucas,MX,22.8909,-109.9124,42,No hotel found
272,agat village,GU,13.3886,144.6585,89,No hotel found
282,bandar abbas,IR,27.1865,56.2808,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE